In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 📦 INSTALAR DEPENDENCIAS
!pip install pokemontcgsdk pandas tqdm

In [ ]:
import requests

url = "https://api.pokemontcg.io/v2/cards"
headers = {"X-Api-Key": "49c41b43-69f9-4fe8-b444-0dee2387f08d"}

response = requests.get(url, params={"pageSize": 1}, headers=headers)
print("Status Code:", response.status_code)
print("JSON:", response.json())


Status Code: 200
JSON: {'data': [{'id': 'dp3-1', 'name': 'Ampharos', 'supertype': 'Pokémon', 'subtypes': ['Stage 2'], 'level': '52', 'hp': '130', 'types': ['Lightning'], 'evolvesFrom': 'Flaaffy', 'abilities': [{'name': 'Jamming', 'text': "After your opponent plays a Supporter card from his or her hand, put 1 damage counter on each of your opponent's Pokémon. You can't use more than 1 Jamming Poké-Body each turn.", 'type': 'Poké-Body'}], 'attacks': [{'name': 'Cluster Bolt', 'cost': ['Lightning', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '70', 'text': "You may discard all Lightning Energy attached to Ampharos. If you do, this attack does 20 damage to each of your opponent's Benched Pokémon that has any Energy cards attached to it. (Don't apply Weakness and Resistance for Benched Pokémon.)"}], 'weaknesses': [{'type': 'Fighting', 'value': '+30'}], 'resistances': [{'type': 'Metal', 'value': '-20'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless'], 'convertedRetre

In [ ]:
from pokemontcgsdk import Set

# Obtener todos los sets
sets = Set.all()

# Contar e imprimir
print("🗂 Todos los sets disponibles:\n")

sets_mas_250 = []

for s in sets:
    total = getattr(s, "total", 0)
    print(f"{s.id} | {s.name} ({s.series}) → total: {total}")

    if total > 250:
        sets_mas_250.append({
            "id": s.id,
            "name": s.name,
            "series": s.series,
            "total": total
        })

print("\n📊 Sets con más de 250 cartas:", len(sets_mas_250))


🗂 Todos los sets disponibles:

base2 | Jungle (Base) → total: 64
ecard2 | Aquapolis (E-Card) → total: 182
base6 | Legendary Collection (Other) → total: 110
ex1 | Ruby & Sapphire (EX) → total: 109
ex2 | Sandstorm (EX) → total: 100
ex4 | Team Magma vs Team Aqua (EX) → total: 97
ex5 | Hidden Legends (EX) → total: 102
pop1 | POP Series 1 (POP) → total: 17
ex7 | Team Rocket Returns (EX) → total: 111
pop2 | POP Series 2 (POP) → total: 17
pop3 | POP Series 3 (POP) → total: 17
pop4 | POP Series 4 (POP) → total: 17
ex15 | Dragon Frontiers (EX) → total: 101
pop5 | POP Series 5 (POP) → total: 17
dpp | DP Black Star Promos (Diamond & Pearl) → total: 56
dp2 | Mysterious Treasures (Diamond & Pearl) → total: 124
dp3 | Secret Wonders (Diamond & Pearl) → total: 132
dp4 | Great Encounters (Diamond & Pearl) → total: 106
pop7 | POP Series 7 (POP) → total: 17
dp5 | Majestic Dawn (Diamond & Pearl) → total: 100
pop9 | POP Series 9 (POP) → total: 17
pl4 | Arceus (Platinum) → total: 111
ru1 | Pokémon Rumble (O

In [ ]:
# 📦 Instalar dependencias
!pip install pokemontcgsdk tqdm --quiet

# 🔌 Montar Google Drive (si aún no está montado)
from google.colab import drive
drive.mount('/content/drive')

# 📁 Ruta de salida en Google Drive
output_dir = "/content/drive/MyDrive/Proyecto API"
import os
os.makedirs(output_dir, exist_ok=True)

# 🧩 Configurar API Key y cargar librerías del SDK
from pokemontcgsdk import RestClient, Card, Set
RestClient.configure('49c41b43-69f9-4fe8-b444-0dee2387f08d')

# 📚 Importar librerías adicionales
import pandas as pd
from tqdm.notebook import tqdm

# 🧠 Función para extraer los datos útiles de cada carta
def extract_card_data(card):
    data = {
        "id": card.id,
        "name": card.name,
        "supertype": card.supertype,
        "subtypes": ", ".join(card.subtypes) if card.subtypes else None,
        "rarity": card.rarity,
        "set_id": card.set.id if card.set else None,
        "set_name": card.set.name if card.set else None,
        "number": card.number,
        "artist": card.artist,
        "types": ", ".join(card.types) if card.types else None,
        # Se accede al 'name' del ancientTrait usando notación de atributo
        "ancientTrait": card.ancientTrait.name if card.ancientTrait else None,
        "legalities": card.legalities.__dict__ if card.legalities else None,
        "images_small": card.images.small if card.images else None,
        "images_large": card.images.large if card.images else None,
    }

    # 💵 TCGPlayer Info (USD)
    if card.tcgplayer:
        data["tcgplayer_url"] = card.tcgplayer.url
        data["tcgplayer_updatedAt"] = card.tcgplayer.updatedAt
        if hasattr(card.tcgplayer, 'prices') and card.tcgplayer.prices is not None:
            tcg_prices = card.tcgplayer.prices.__dict__
            for price_type, price_info in tcg_prices.items():
                if isinstance(price_info, dict):
                    for price_field, price_value in price_info.items():
                        data[f"tcg_{price_type}_{price_field}"] = price_value

    # 💶 Cardmarket Info (EUR)
    if hasattr(card, 'cardmarket') and card.cardmarket:
        data["cardmarket_url"] = card.cardmarket.url
        data["cardmarket_updatedAt"] = card.cardmarket.updatedAt
        if hasattr(card.cardmarket, 'prices') and card.cardmarket.prices is not None:
            cm_prices = card.cardmarket.prices.__dict__
            for price_key, price_value in cm_prices.items():
                data[f"cm_{price_key}"] = price_value

    return data

# 🔄 Función para obtener todas las cartas de un set (paginación)
def get_all_cards_for_set(set_id):
    cards = []
    page = 1
    while True:
        current_page = Card.where(q=f"set.id:{set_id}", page=page, pageSize=250)
        if not current_page:
            break
        cards.extend(current_page)
        page += 1
    return cards

# 📚 Obtener todos los sets
sets = Set.all()
all_cards_data = []

# 🔄 Procesar cada set y guardar su CSV
for s in tqdm(sets, desc="Sets procesados"):
    print(f"\n📦 Procesando set: {s.name}")
    set_cards = get_all_cards_for_set(s.id)
    set_data = []

    for card in tqdm(set_cards, desc=f"  Cartas en {s.name}", leave=False):
        card_info = extract_card_data(card)
        set_data.append(card_info)
        all_cards_data.append(card_info)

    # 💾 Guardar CSV individual del set (se usa un nombre de archivo sin espacios)
    df_set = pd.DataFrame(set_data)
    filename_set = f"{output_dir}/{s.id}_{s.name.replace(' ', '_')}.csv"
    df_set.to_csv(filename_set, index=False)

# 💾 Guardar CSV combinado de todas las cartas
df_all = pd.DataFrame(all_cards_data)
df_all.to_csv(f"{output_dir}/all_cards_combined.csv", index=False)

print("\n✅ Todo listo. Archivos CSV guardados en tu Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Sets procesados:   0%|          | 0/165 [00:00<?, ?it/s]


📦 Procesando set: Jungle


  Cartas en Jungle:   0%|          | 0/64 [00:00<?, ?it/s]


📦 Procesando set: Aquapolis


...


📦 Procesando set: Prismatic Evolutions


  Cartas en Prismatic Evolutions:   0%|          | 0/180 [00:00<?, ?it/s]


📦 Procesando set: Journey Together


  Cartas en Journey Together:   0%|          | 0/190 [00:00<?, ?it/s]


✅ Todo listo. Archivos CSV guardados en tu Google Drive.


In [3]:
# 📦 Instalar dependencias
!pip install pokemontcgsdk tqdm --quiet

# 🔌 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📁 Rutas
base_dir = "/content/drive/MyDrive/Proyecto API"
meta_dir = f"{base_dir}/Metadatos Cartas"
price_dir = f"{base_dir}/MonthlyCSV"
combo_dir = f"{base_dir}/Backup Combinado"

import os
for path in [meta_dir, price_dir, combo_dir]:
    os.makedirs(path, exist_ok=True)

# 🧩 Configurar API Key
from pokemontcgsdk import RestClient, Card, Set
RestClient.configure('49c41b43-69f9-4fe8-b444-0dee2387f08d')

# 📚 Librerías
import pandas as pd
from tqdm.notebook import tqdm
from datetime import date

# 🧠 Extraer datos
def extract_card_data(card):
    metadata = {
        "id": card.id,
        "name": card.name,
        "supertype": card.supertype,
        "subtypes": ", ".join(card.subtypes) if card.subtypes else None,
        "rarity": card.rarity,
        "set_id": card.set.id if card.set else None,
        "set_name": card.set.name if card.set else None,
        "number": card.number,
        "artist": card.artist,
        "types": ", ".join(card.types) if card.types else None,
        "ancientTrait": card.ancientTrait.name if card.ancientTrait else None,
        "legalities": card.legalities.__dict__ if card.legalities else None,
        "images_small": card.images.small if card.images else None,
        "images_large": card.images.large if card.images else None,
    }

    prices = {}

    # ✅ TCGPlayer
    if card.tcgplayer:
        prices["tcgplayer_url"] = card.tcgplayer.url
        prices["tcgplayer_updatedAt"] = card.tcgplayer.updatedAt
        if hasattr(card.tcgplayer, 'prices') and card.tcgplayer.prices is not None:
            tcg_prices = getattr(card.tcgplayer.prices, "__dict__", {})
            for ptype, info in tcg_prices.items():
                if isinstance(info, dict):
                    for k, v in info.items():
                        prices[f"tcg_{ptype}_{k}"] = v

    # ✅ Cardmarket
    if hasattr(card, 'cardmarket') and card.cardmarket:
        prices["cardmarket_url"] = card.cardmarket.url
        prices["cardmarket_updatedAt"] = card.cardmarket.updatedAt
        if hasattr(card.cardmarket, 'prices') and card.cardmarket.prices is not None:
            cm_prices = getattr(card.cardmarket.prices, "__dict__", {})
            for k, v in cm_prices.items():
                prices[f"cm_{k}"] = v

    return metadata, prices

# 🔄 Obtener cartas de un set
def get_all_cards_for_set(set_id):
    cards, page = [], 1
    while True:
        current = Card.where(q=f"set.id:{set_id}", page=page, pageSize=250)
        if not current:
            break
        cards.extend(current)
        page += 1
    return cards

# 📥 Procesar sets
sets = Set.all()
meta_all, prices_all, combined_all = [], [], []
today_str = str(date.today())

for s in tqdm(sets, desc="Procesando sets"):
    print(f"\n📦 Set: {s.name}")
    cards = get_all_cards_for_set(s.id)

    for card in tqdm(cards, desc=f"  Cartas en {s.name}", leave=False):
        meta, price = extract_card_data(card)
        meta_all.append(meta)
        prices_all.append({**{"id": meta["id"], "name": meta["name"], "set_id": meta["set_id"], "set_name": meta["set_name"]}, **price})
        combined_all.append({**meta, **price})

# 💾 Guardar CSVs
pd.DataFrame(meta_all).to_csv(f"{meta_dir}/metadata_cards.csv", index=False)
print("✅ Metadatos guardados")

pd.DataFrame(prices_all).to_csv(f"{price_dir}/all_cards_prices_{today_str}.csv", index=False)
print("✅ Precios guardados")

pd.DataFrame(combined_all).to_csv(f"{combo_dir}/all_cards_combined_{today_str}.csv", index=False)
print("✅ Combinado guardado")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Procesando sets:   0%|          | 0/165 [00:00<?, ?it/s]


📦 Set: Base


  Cartas en Base:   0%|          | 0/102 [00:00<?, ?it/s]


📦 Set: Jungle


...


📦 Set: Prismatic Evolutions


  Cartas en Prismatic Evolutions:   0%|          | 0/180 [00:00<?, ?it/s]


📦 Set: Journey Together


  Cartas en Journey Together:   0%|          | 0/190 [00:00<?, ?it/s]

✅ Metadatos guardados
✅ Precios guardados
✅ Combinado guardado
